In [24]:
import math, os, sys
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime
from scipy import spatial
import densitySpaceTime.settings as sett 
import densitySpaceTime.kde as kde
from tqdm import tqdm, trange
from tqdm.contrib import itertools

# Setting
settings.py

- p1 = spatial bandwidth
- p2 = temporal bandwidth
- p3 = spatial resolution
- p4 = temporal resolution
- npts = number of input points
- ct1 = constant term 1 for density equation
- ct2 = constant term 2 for density equation

In [17]:
# Initialize
sett.init()

#read parameters
parameters = {'p1' : 1000.0, 'p2' : 24.0, 'p3' : 3.0, 'p4' : 3.0}

sett.p1 = float(parameters['p1'])	# p1 = spatial bandwidth(meter)
sett.p2 = float(parameters['p2'])	# p2 = temporal bandwidth(hour)
sett.p3 = float(parameters['p3'])	# p3 = spatial resolution
sett.p4 = int(parameters['p4'])	    # p4 = temporal resolution

In [18]:
# Load Data
df = gpd.read_file('data/gdf_crime.gpkg')
df = df.to_crs(epsg=5181) # 좌표계 m단위로 변경
df = df[['RECV_CPLT_DM', 'geometry']]
df.RECV_CPLT_DM = df.RECV_CPLT_DM.dt.to_period('H')
df.RECV_CPLT_DM = df.RECV_CPLT_DM.dt.to_timestamp()

# data에 time index 찾아 넣기(기준시점으로부터 시간)

df['T'] = 0.0
for i in range(len(df)):
    # 2021년 1월 1일 0시 기준
    delta = df.RECV_CPLT_DM[i] - pd.Timestamp(2020, 1, 1, 0) 
    df.iat[i, 2] = delta / pd.Timedelta(hours=1)

# X, Y 좌표 추출
df['X'] = df.geometry.x
df['Y'] = df.geometry.y

df.head(3) # ex

,RECV_CPLT_DM,geometry,T,X,Y
0,2020-01-02 09:00:00,POINT (209315.288 371437.494),33.0,209315.288434,371437.493906
1,2020-01-02 09:00:00,POINT (201952.949 363622.909),33.0,201952.948717,363622.908901
2,2020-01-02 10:00:00,POINT (148880.940 383353.530),34.0,148880.940462,383353.529568


In [19]:
# 각 벡터 분리
inX = df['X']
inY = df['Y']
inT = df['T']
inXY = zip(inX, inY)

# X,Y 및 T의 범위(최대, 최소) 확인
grid = gpd.read_file('data/grid.gpkg').to_crs(epsg=5181)
xmin, ymin, xmax, ymax = grid.total_bounds
tmin, tmax = 0.0, max(inT)

In [20]:
# 전역변수 설정(global variables)
sett.npts = len(df)
sett.ct1 = 0.5 * math.pi
sett.ct2 = pow(10.0, 5) / (sett.npts * pow(sett.p1, 2) * sett.p2)

In [21]:
# STKDE 연산

xminDiff = xmin%sett.p3
xmaxDiff = xmax%sett.p3
yminDiff = ymin%sett.p3
ymaxDiff = ymax%sett.p3
tminDiff = tmin%sett.p4
tmaxDiff = tmax%sett.p4

xminP = xmin - xminDiff + sett.p3
xmaxP = xmax - xmaxDiff + sett.p3
yminP = ymin - yminDiff + sett.p3
ymaxP = ymax - ymaxDiff + sett.p3
tminP = tmin - tminDiff + sett.p4
tmaxP = tmax - tmaxDiff + sett.p4

In [ ]:
from tqdm import trange
np.arange()

In [15]:
# grids

xyGrid = []
for i, j in itertools.product(np.arange(int(xminP),int(xmaxP),sett.p3), np.arange(int(yminP),int(ymaxP),sett.p3)):
    xyGrid.append([i,j])

tGrid = []
for k in tqdm(range(int(tminP),int(tmaxP),sett.p4)):
    tGrid.append([k])

  1%|▏         | 920/63000 [00:34<39:08, 26.43it/s]  


KeyboardInterrupt: 

In [ ]:
#build trees
stree = spatial.cKDTree(inXY)
ttree = spatial.cKDTree(inT)